# Mark Corpus

In [1]:
import json 

In [42]:
dataset = json.load(open("./datasets/covidqa/dev-v1.1.json"))

In [43]:
out_file = open("./context_dev.txt","w+")

In [44]:
for data in dataset["data"]:
    for para in data['paragraphs']:
        context = para["context"]
        out_file.write(context)
        out_file.write("\n")
out_file.close()

In [34]:
data

{'paragraphs': [{'qas': [{'question': 'Is the origin and epidemiology of the 1918 swine flu (Spanish Influenza) known?',
     'id': '1075',
     'answers': [{'text': 'ongoing studies to map Virulence\nfactors are yielding interesting results. The 1918 sequence\ndata, however, leave unanswered questions about the ori-\ngin of the Virus (19) and about the epidemiology of the\npandemic.',
       'answer_start': 5741}],
     'is_impossible': False},
    {'question': 'How dangerous are the modern H1N1 (swine flu) and the H3N2 (Influenza A) viruses compared  to the 1918 H1N1 (swine flu Spanish Influenza)  viruses?',
     'id': '1072',
     'answers': [{'text': 'the human H1N1 and H3N2 lin-\neages have both been associated with substantially lower\nrates ofillness and death than the virus of 1918. In fact, cur-\nrent H1N1 death rates are even lower than those for H3N2\nlineage strains (prevalent from 1968 until the present).',
       'answer_start': 3781}],
     'is_impossible': False},
    {

# mlm prepare 

In [1]:
from tasks.squad.train import BertForQA
import torch

In [2]:
from transformers import RobertaModel

In [5]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
from pytorch_lightning import Trainer

In [17]:
mlm_model = RobertaForMaskedLM.from_pretrained('./cached_models/roberta-base')

In [5]:
model = BertForQA.load_from_checkpoint(
        checkpoint_path="./cached_models/roberta_squad1_2epoch/roberta_squad1_2epoch.ckpt",
        hparams_file="./outputs/roberta-base/squad1/lightning_logs/squad1_squad1/hparams.yaml",
        map_location=None)

/opt/anaconda/envs/covidqa/lib/python3.8/site-packages/pytorch_lightning/core/saving.py:335: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  tags = yaml.load(fp)
Some weights of the model checkpoint at /home/xiangpan/Labs/CovidQA/multi_task_qa/cached_models/roberta-base were not used when initializing BertForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that

In [33]:
model.tokenizer._save_pretrained(True,"./cached_models/roberta_mlm/")

Vocabulary path (True) should be a directory


TypeError: can only concatenate str (not "NoneType") to str

In [6]:
mlm_model.roberta.load_state_dict(model.model.roberta.state_dict())

<All keys matched successfully>

In [18]:
mlm_model.save_pretrained("./cached_models/roberta_mlm/")

# Convert MLM to Task Model

In [13]:
import torch
from tasks.squad.train import BertForQA

In [10]:
from transformers import RobertaTokenizer, RobertaForMaskedLM

In [11]:
mlm_model = RobertaForMaskedLM.from_pretrained('./cached_models/roberta_squad1_2epoch_covidmlm_3epoch/')

In [12]:
model = BertForQA.load_from_checkpoint(
        checkpoint_path="./cached_models/roberta_squad1_2epoch/roberta_squad1_2epoch.ckpt",
        hparams_file="./outputs/roberta-base/squad1/lightning_logs/squad1_squad1/hparams.yaml",
        map_location=None)

Some weights of the model checkpoint at /home/xiangpan/Labs/CovidQA/multi_task_qa/cached_models/roberta-base were not used when initializing BertForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at /home/xiangpan/Labs/CovidQA/multi_task_qa/cached_model

In [13]:
trainer = Trainer()

GPU available: True, used: False
INFO:lightning:GPU available: True, used: False
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores


In [14]:
model.model.roberta.load_state_dict(mlm_model.roberta.state_dict())

<All keys matched successfully>

In [17]:
torch.save(model.state_dict(),'./cached_models/roberta_squad1_2epoch_covidmlm_3epoch/roberta_squad1_2epoch_covidmlm_3epoch.ckpt')

In [16]:
model

BertForQA(
  (model): BertForQuestionAnswering(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear(in_features=7

In [13]:
trainer = Trainer(default_root_dir="./cached_models/roberta_squad1_2epoch/roberta_squad1_2epoch.ckpt")

GPU available: True, used: False
INFO:lightning:GPU available: True, used: False
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
/opt/anaconda/envs/covidqa/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)


In [17]:
trainer.keys()

AttributeError: 'Trainer' object has no attribute 'keys'

# Convert Text Classification To QA Model

In [6]:
import torch
from tasks.tnews.train import TNewsClassificationTask

In [3]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [4]:
RobertaForSequenceClassification.from_pre_trained_

TypeError: load_state_dict() missing 2 required positional arguments: 'self' and 'state_dict'

In [11]:
cls_model = TNewsClassificationTask.load_from_checkpoint(
        checkpoint_path="./outputs/roberta_squad1_2epoch/covidq_cls/epoch=3_v0.ckpt",
        hparams_file="outputs/roberta_squad1_2epoch/covidq_cls/lightning_logs/version_0/hparams.yaml",
        map_location=None)

Some weights of the model checkpoint at /home/xiangpan/Labs/CovidQA/multi_task_qa/cached_models/roberta_squad1_2epoch were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /home/xiangpan/Labs/CovidQA/multi_task_qa/cached_

In [10]:
model.model.roberta.save_pre_trained

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [14]:
model = BertForQA.load_from_checkpoint(
        checkpoint_path="./cached_models/roberta_squad1_2epoch/roberta_squad1_2epoch.ckpt",
        hparams_file="./outputs/roberta-base/squad1/lightning_logs/squad1_squad1/hparams.yaml",
        map_location=None)

Some weights of the model checkpoint at /home/xiangpan/Labs/CovidQA/multi_task_qa/cached_models/roberta-base were not used when initializing BertForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at /home/xiangpan/Labs/CovidQA/multi_task_qa/cached_model

In [17]:
model.model.roberta.load_state_dict(cls_model.model.roberta.state_dict())

<All keys matched successfully>

In [19]:
torch.save(model.state_dict(),'./cached_models/roberta_squad1_2epoch_covidqcls_3epoch/roberta_squad1_2epoch_covidqcls_3epoch.ckpt')